# Notebook to test the Fit of SEDs to an IGMF model

Inspired from https://github.com/woodmd/haloanalysis/blob/master/haloanalysis/scripts/fit_igmf.py#L65-L66

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2

## Imports

In [4]:
import os
import sys
from collections import OrderedDict

import yaml

import numpy as np
from astropy.io import fits
from astropy.table import Table, Column, join, hstack, vstack
import matplotlib.pyplot as plt
import matplotlib

import fermipy.utils as utils
from fermipy.spectrum import *
from fermipy.castro import CastroData

from haloanalysis.utils import create_mask, load_source_rows
from haloanalysis.sed import HaloSED
from haloanalysis.model import CascModel, CascLike
from haloanalysis.model import scan_igmf_likelihood

## load the catalog

Catalog file:

In [5]:
cat = '../data/table_std_psf0123_joint2a_stdmodel_lnl_v14.fits'

In [6]:
tab_pars = Table.read(cat,hdu='SCAN_PARS')
tab_ebounds = Table.read(cat,hdu='EBOUNDS')

In [7]:
tables = [Table.read(cat,'CATALOG'),
            Table.read(cat,'LIKELIHOOD'),
            Table.read(cat,'SED')]

In [8]:
for i, t in enumerate(tables):
    if 'NAME' in t.columns:
        t['name'] = t['NAME']

In [9]:
print tables[0]['name'].data
print tables[1]['name'].data
print tables[2]['name'].data

['FHES J2359.8-3736' 'FHES J0001.2-0746' 'FHES J0001.5+2113' ...,
 'FHES J0538.0-6913' 'FHES J0535.6-6734' 'FHES J0524.7-6937']
['3FGL J0000.2-3738' '3FGL J0001.2-0748' '3FGL J0001.4+2120' ..., 'LMC P2'
 'LMC P3' 'LMC P4']
['3FGL J0000.2-3738' '3FGL J0001.2-0748' '3FGL J0001.4+2120' ..., 'LMC P2'
 'LMC P3' 'LMC P4']


Remove identifier so in names so that join operation works

In [10]:
for i, t in enumerate(tables):
    for j,n in enumerate(t['name']):
        tables[i]['name'][j] = n[5:-2] ### WORK AROUND

In [11]:
print tables[0]['name'].data

['J2359.8-37' 'J0001.2-07' 'J0001.5+21' ..., 'J0538.0-69' 'J0535.6-67'
 'J0524.7-69']


### join the tables -- DOES ONLY WORK WITH WORKAROUND ABOVE

In [12]:
tab_casc = join(tables[0],tables[1]) # table is empty since 'name' names are different
tab_casc = join(tab_casc,tables[2])

In [13]:
print tab_casc['name'].data.shape

(638,)


Load the TeV SEDs

In [14]:
tab_sed_tev = Table.read('../data/CompiledTeVSources.fits')

In [15]:
print tab_sed_tev

 3FGL_NAME             SOURCE_FULL            ...   NORM_ERRN [25]  
------------ -------------------------------- ... ------------------
J0232.8+2016       1ES0229+200_HESS_2005-2006 ...  1.3277e-08 .. nan
J0232.8+2016    1ES0229+200_VERITAS_2009-2012 ...     2.8e-08 .. nan
J0349.2-1158      1ES0347-121_HESS_2006-09-12 ...  3.5758e-08 .. nan
J0416.8+0104       1ES0414+009_HESS_2005-2009 ...   1.017e-07 .. nan
J0416.8+0104    1ES0414+009_VERITAS_2008-2011 ...    1.09e-08 .. nan
J0809.8+5218    1ES0806+524_VERITAS_2006-2008 ... 9.29454e-08 .. nan
J1015.0+4925           1ES1011+496_MAGIC_2007 ...    1.69e-06 .. nan
J1103.5-2329       1ES1101-232_HESS_2004-2005 ...  1.0851e-07 .. nan
J1217.8+3007           1ES1215+303_MAGIC_2011 ...    1.53e-06 .. nan
J1217.8+3007    1ES1215+303_VERITAS_2008-2012 ...  5.7133e-08 .. nan
         ...                              ... ...                ...
J2158.8-3013      PKS2155-304_HESS_2006-08-03 ...   2.947e-07 .. nan
J2158.8-3013      PKS2155-304_HESS

### Load the IGMF model cube

In [17]:
casc_model = CascModel.create_from_fits(
    '/nfs/farm/g/glast/u/mmeyer/projects/FermiHalo/Output/EBLm6/th_jet6.00/gam-2.00/results_merged_z_th6d_t1e7.fits',
)

/afs/slac.stanford.edu/u/gl/mmeyer/projects/python/haloanalysis/haloanalysis/model.py:809: RuntimeWarning: invalid value encountered in divide
  data_casc_flux = np.array(tab_casc_flux/tab_inj_flux[..., np.newaxis, np.newaxis])
/afs/slac.stanford.edu/u/gl/mmeyer/projects/python/haloanalysis/haloanalysis/model.py:810: RuntimeWarning: invalid value encountered in divide
  data_prim_flux = np.array(tab_prim_flux/tab_inj_flux)


In [20]:
casc_model.set_eblmodel(eblmodel = 'dominguez')

### Loop over the sources and perform the likelihood fit

Spectra you want to include in the analysis

In [21]:
src_names = ['1ES0229+200_HESS_2005-2006']

In [22]:
nstep = 5
casc_scale = 1.
casc_r68_scale = 1.

In [23]:
tab_igmf = []

for name in src_names:

    rows_sed_tev = load_source_rows(tab_sed_tev, [name], key='SOURCE_FULL')
    #cat_names = [ 'FHES %s'%row['3FGL_NAME'] for row in rows_sed_tev ]
    cat_names = [ '%s'%row['3FGL_NAME'][:-2] for row in rows_sed_tev ] # WORK AROUND
    
    cat_names = np.unique(np.array(cat_names))
    rows_sed_gev = load_source_rows(tab_casc, cat_names, key='name')
    rows_casc = load_source_rows(tab_casc, cat_names, key='name')
    
    print rows_sed_tev
    print rows_sed_gev
    print rows_casc
    
    tab = scan_igmf_likelihood(casc_model, rows_sed_tev, rows_sed_gev,
                                rows_casc, tab_pars, tab_ebounds, nstep,
                                casc_scale, casc_r68_scale)
    #tab_igmf += [tab]

 3FGL_NAME          SOURCE_FULL         ...   NORM_ERRP [25]    NORM_ERRN [25] 
------------ -------------------------- ... ----------------- -----------------
J0232.8+2016 1ES0229+200_HESS_2005-2006 ... 1.3277e-08 .. nan 1.3277e-08 .. nan
   name         codename     ...       dloglike_scan [20,9]     
                             ...                                
---------- ----------------- ... -------------------------------
J0232.8+20 3fgl_j0232.8+2016 ... -4.2029670149 .. -2.70594689762
   name         codename     ...       dloglike_scan [20,9]     
                             ...                                
---------- ----------------- ... -------------------------------
J0232.8+20 3fgl_j0232.8+2016 ... -4.2029670149 .. -2.70594689762
((0, 0), -65.160357820170816, [0.14000000000000001, -4.0, -20.0], [1.7781826364287957e-13, -1.6174113205437628, 4929568.6279378785])
((0, 1), -66.836542031590739, [0.14000000000000001, -4.0, -18.0], [1.7762412008607477e-13, -1.6170279304050

((0, 0), -65.160357820170816, [0.14000000000000001, -4.0, -20.0], [1.7781826364287957e-13, -1.6174113205437628, 4929568.6279378785])